In [1]:
import random
import pprint
import json
from elasticsearch import Elasticsearch

In [2]:
def frandom2d():
    return float("%0.3f"%(100*random.random()))

def frandom3d():
    return float("%0.3f"%(1000*random.random()))

In [3]:
def createcarding(oneday):
    for i in range(6):
        carding_temp = {
            "machine_id": "C"+str(i+1),
            "machine_type": "CARDING",
            "cur_production": frandom3d(),
            "running_hours": 6.6,
            "production_breakup": [
              {
                "color": "black",
                "cur_production": frandom3d(),
                "number_of_spinbox": 240
              },
              {
                "color": "FO",
                "cur_production": frandom3d(),
                "number_of_spinbox": 240
              }
            ],
            "eb_details": {
              "number_of_brakes": 5,
              "units": frandom2d()
            }
        }
        if i<4:
            carding_temp["godown"]= "A"
        else:
            carding_temp["godown"]="D"
        oneday["machine_production_details"].append(carding_temp)
    return    

In [4]:
def createblowroom(oneday):
    for i in range(2):
        blowroom_temp= {
            "machine_id": "BR"+str(i+1),
            "machine_type": "BLOWROOM",
            "eb_details": {
              "units": frandom2d()
            },
            "waste_details": {
              "mbo_waste": frandom2d(),
              "sweeping_waste": frandom2d(),
              "filter_waste": frandom2d(),
              "powder_waste": frandom2d()
            }
        }
        blowroom_temp["waste_details"]["total_waste"]=blowroom_temp["waste_details"]["mbo_waste"]+\
            blowroom_temp["waste_details"]["sweeping_waste"]+\
            blowroom_temp["waste_details"]["filter_waste"]+\
            blowroom_temp["waste_details"]["powder_waste"]
        if i==0:
            blowroom_temp["godown"]="A"
        else:
            blowroom_temp["godown"]="D"
        oneday["machine_production_details"].append(blowroom_temp)
    return

In [5]:
def createdrawing(oneday):
    for i in range(2):
        drawing_temp={
        "machine_id": "DM"+str(i+1),
        "machine_type": "DRAWING",
        "cur_production": frandom3d(),
        "running_hours": 7.5,
        "godown": "B",
        "waste_details": {
          "fan_waste": frandom2d(),
          "sweeping_waste": frandom2d(),
          "slaver_waste": frandom2d()
        },
        "eb_details": {
          "units": frandom2d(),
          "number_of_brakes": 5
            }
        }
        
        oneday["machine_production_details"].append(drawing_temp)

In [6]:
def createOE(oneday):
    for i in range(3):
        OE_temp={
        "machine_id": "OE"+str(i+1),
        "machine_type": "OE",
        "running_hours": 5.5,
        "godown": "B",
        "color_details": [
          {
            "color": "black",
            "cur_production": frandom3d(),
            "number_of_spinbox": 240,
          },
          {
            "color": "FO",
            "cur_production": frandom3d(),
            "number_of_spinbox": 240,
          }
        ],
        "waste_details": {
          "rotor_waste": frandom2d(),
          "yarn_waste": frandom2d(),
          "slaver_waste": frandom2d()
        },
        "eb_details": {
          "number_of_brakes": 5,
          "units": frandom2d()
            }
        }
        oneday["machine_production_details"].append(OE_temp)
        

In [7]:
def createTFO(oneday):
    for i in range(3):
        tfo_temp = {
        "machine_id": "TFO"+str(i+1),
        "machine_type": "TFO",
        "production_details": [
          {
            "count": 20,
            "color": "blue",
            "godown":"C",
            "cur_production": frandom3d(),
            "running_hours": 7.5,
            "sider": "A",
            "sider_name": "Ram",
            "waste_details": {
              "yarn_waste": frandom2d()
                }
            },
            {
            "count": 20,
            "color": "green",
            "cur_production": frandom3d(),
            "running_hours": 7.5,
            "sider": "B",
            "sider_name": "Raja",
            "waste_details": {
              "yarn_waste": frandom2d()
                }
              }
            ],
        "eb_details": {
        "units": frandom2d()
            },
        "common_waste_details": {
        "sweep_waste": frandom2d()
            }
        }
        oneday["machine_production_details"].append(tfo_temp)

In [8]:
def createcheesewinding(oneday):
    cheese_winding_temp={
            "count": 20,
            "color": "blue",
            "machine_id": "CW"+str(i+1),
            "machine_type": "CHEESEWINDING",
            "godown": "B",
            "production_details": [
              {
                "cur_production": frandom3d(),
                "running_hours": 7.5,
                "sider": "A",
                "sider_name": "Ram",
                "waste_details": {
                "yarn_waste":  frandom2d()
                }
              },
              {
                "count": 20,
                "color": "green",
                "cur_production":  frandom3d(),
                "running_hours": 7.5,
                "sider": "B",
                "sider_name": "Raja",
                "waste_details": {
                  "yarn_waste":  frandom2d()
                }
              }
            ],
            "eb_details": {
              "units":  frandom2d()
            },
            "common_waste_details": {
              "sweep_waste":  frandom2d()
            }
    }
    oneday["machine_production_details"].append(cheese_winding_temp)

In [9]:
def calc_waste_percentage(oneday):
    temp_machines=oneday["machine_production_details"]
    A_godown_carding_pdn=0
    D_godown_carding_pdn=0
    for i in temp_machines:
        if i["machine_type"]=="BLOWROOM":
            if i["godown"]=="A":
                A_godown_BR_waste=i["waste_details"]["total_waste"]
            if i["godown"]=="D":
                D_godown_BR_waste=i["waste_details"]["total_waste"]
        if i["machine_type"]=="CARDING":
            if i["godown"]=="A":
                A_godown_carding_pdn+=i["cur_production"]
            if i["godown"]=="D":
                D_godown_carding_pdn+=i["cur_production"]
    total_waste_A_godown=oneday["carding_waste_details"][0]["total_waste"] + A_godown_BR_waste
    total_waste_D_godown=oneday["carding_waste_details"][1]["total_waste"] +D_godown_BR_waste
    percentage_waste_gen_A= A_godown_BR_waste / (A_godown_carding_pdn + A_godown_BR_waste )
    percentage_waste_gen_D=D_godown_BR_waste / (D_godown_carding_pdn + D_godown_BR_waste )
    oneday["overall_waste_details"]={"A_godown_waste_percentage":percentage_waste_gen_A,"D_godown_waste_percentage":percentage_waste_gen_D}

In [10]:
es = Elasticsearch([{'host':'localhost', 'port':9200}])
oneday=dict()
monthdict={}
randomPdn=[]
grandtotal=0

for i in range(10):
    grandtotal+=sum(randomPdn)
    oneday={
    "date": str(random.randint(1,30))+'.'+str(random.randint(1,12))+'.'+'2019',
	"color": random.choice(['red','green','blue']),
	"count": random.choice([10,20,30]),
	"kgs": random.choice([1000,2000,3000,4000,5000]),
	"month_of_total": "",
	"labor": random.choice([1,2]),
	"shift": random.choice(['A','B','C']),
    "carding_waste_details": [{
    "godown":"A",
	"flat_waste": frandom2d(),
	"sweep_waste": frandom2d()
	},
    {
    "godown":"D",
	"flat_waste": frandom2d(),
	"sweep_waste": frandom2d()
    }],
    "machine_production_details": list()
}
    createcarding(oneday)
    createblowroom(oneday)
    createdrawing(oneday)
    createOE(oneday)
    createTFO(oneday)
    createcheesewinding(oneday)
    oneday["carding_waste_details"][0]["total_waste"]=oneday["carding_waste_details"][0]["flat_waste"]+\
                                            oneday["carding_waste_details"][0]["sweep_waste"]
    oneday["carding_waste_details"][1]["total_waste"]=oneday["carding_waste_details"][1]["flat_waste"]+\
                                            oneday["carding_waste_details"][1]["sweep_waste"]
    calc_waste_percentage(oneday)
	
    #es.index(index='oe-index', doc_type='oe-prdn', id=i+1,body=json.dumps(oneday))
    

In [107]:
print (json.dumps(oneday))

{"date": "28.1.2019", "color": "green", "count": 20, "kgs": 1000, "month_of_total": "", "labor": 2, "shift": "B", "carding_waste_details": [{"godown": "A", "flat_waste": 45.497, "sweep_waste": 61.56, "total_waste": 107.057}, {"godown": "D", "flat_waste": 35.455, "sweep_waste": 96.536, "total_waste": 131.99099999999999}], "machine_production_details": [{"machine_id": "C1", "machine_type": "CARDING", "cur_production": 706.621, "running_hours": 6.6, "production_breakup": [{"color": "black", "cur_production": 472.824, "number_of_spinbox": 240}, {"color": "FO", "cur_production": 59.29, "number_of_spinbox": 240}], "eb_details": {"number_of_brakes": 5, "units": 76.134}, "godown": "A"}, {"machine_id": "C2", "machine_type": "CARDING", "cur_production": 808.286, "running_hours": 6.6, "production_breakup": [{"color": "black", "cur_production": 415.438, "number_of_spinbox": 240}, {"color": "FO", "cur_production": 919.497, "number_of_spinbox": 240}], "eb_details": {"number_of_brakes": 5, "units": 6